# Лабораторная работа №4 (Проведение исследований со случайным лесом)

### Создание бейзлайна и оценка качества

Импорт необходимых библиотек.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from collections import Counter
import random
from typing import Union, List, Tuple
from sklearn.base import BaseEstimator

Подготовка данных.

In [2]:
path = 'drive/MyDrive/ai_data/'
heart = pd.read_csv(path + 'heart.csv')
heart.shape, heart.columns

((918, 12),
 Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
        'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
        'HeartDisease'],
       dtype='object'))

Посмотрим состав таблицы и наличие пропусков, а также баланс HeartDisease.

In [3]:
display(heart.head())
display(heart.info())
display(heart.describe())
print("Missing per column:\n", heart.isna().sum())
print(heart['HeartDisease'].value_counts(normalize=True))

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


None

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


Missing per column:
 Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64
HeartDisease
1    0.553377
0    0.446623
Name: proportion, dtype: float64


Выполним те же действия для второго датасета.

In [4]:
melbourne = pd.read_csv(path + 'melb_data.csv')
melbourne.shape, melbourne.columns

((13580, 21),
 Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
        'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
        'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
        'Longtitude', 'Regionname', 'Propertycount'],
       dtype='object'))

In [5]:
display(melbourne.head())
display(melbourne.info())
display(melbourne.describe())
print(melbourne.isna().sum())
print(melbourne['Price'].describe())

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

None

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64
count    1.358000e+04
mean     1.075684e+06
std      6.393107e+05
min      8.500000e+04
25%      6.500000e+05
50%      9.030000e+05
75%      1.330000e+06
max      9.000000e+06
Name: Price, dtype: float64


Удалим столбцы с большим количеством пропусков, заполним столбцы с маленьким количеством пропусков медианой и модой

In [6]:
columns_to_drop = ['BuildingArea', 'YearBuilt']
melbourne_clean = melbourne.drop(columns=columns_to_drop)

car_median = melbourne_clean['Car'].median()
melbourne_clean['Car'] = melbourne_clean['Car'].fillna(car_median)

council_mode = melbourne_clean['CouncilArea'].mode()[0]
melbourne_clean['CouncilArea'] = melbourne_clean['CouncilArea'].fillna(council_mode)

print(melbourne_clean.isna().sum())

Suburb           0
Address          0
Rooms            0
Type             0
Price            0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
CouncilArea      0
Lattitude        0
Longtitude       0
Regionname       0
Propertycount    0
dtype: int64


Разделим данные на тренировочную и тестовую выборки:

In [7]:
X_heart = heart.drop('HeartDisease', axis=1)
y_heart = heart['HeartDisease']
X_heart_train, X_heart_test, y_heart_train, y_heart_test = train_test_split(
    X_heart, y_heart, test_size=0.2, random_state=42, stratify=y_heart
)


X_melbourne = melbourne_clean.drop('Price', axis=1)
y_melbourne = melbourne_clean['Price']
X_melbourne_train, X_melbourne_test, y_melbourne_train, y_melbourne_test = train_test_split(
    X_melbourne, y_melbourne, test_size=0.2, random_state=42
)

Проведем обработку категориальных переменных:

In [8]:
X_heart_train_processed = X_heart_train.copy()
X_heart_test_processed = X_heart_test.copy()
X_melbourne_train_processed = X_melbourne_train.copy()
X_melbourne_test_processed = X_melbourne_test.copy()

Обработка категориальных переменных для Heart dataset:

In [9]:
categorical_heart = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
for col in categorical_heart:
    print(f"{col}: {X_heart_train[col].unique()}")

label_encoders_heart = {}
for col in categorical_heart:
    le = LabelEncoder()
    X_heart_train_processed[col] = le.fit_transform(X_heart_train[col])
    X_heart_test_processed[col] = le.transform(X_heart_test[col])
    label_encoders_heart[col] = le

print("\nРезультат:")
print(X_heart_train_processed[categorical_heart].head())
print(f"{X_heart_train_processed[categorical_heart].dtypes}")

Sex: ['M' 'F']
ChestPainType: ['ATA' 'ASY' 'NAP' 'TA']
RestingECG: ['ST' 'Normal' 'LVH']
ExerciseAngina: ['Y' 'N']
ST_Slope: ['Flat' 'Up' 'Down']

Результат:
     Sex  ChestPainType  RestingECG  ExerciseAngina  ST_Slope
485    1              1           2               1         1
486    1              1           2               0         2
117    0              0           2               1         1
361    1              0           1               1         1
296    1              0           1               1         1
Sex               int64
ChestPainType     int64
RestingECG        int64
ExerciseAngina    int64
ST_Slope          int64
dtype: object


Обработка категориальных переменных для Melbourne dataset. Применяем One-Hot Encoding для сложных категориальных признаков. Удаляем нечисловые столбцы, которые не будем использовать.

In [10]:
categorical_melbourne = ['Type', 'Method', 'CouncilArea', 'Regionname']
for col in categorical_melbourne:
    print(f"{col}: {X_melbourne_train[col].nunique()}")

simple_categorical = ['Type', 'Method']
complex_categorical = ['CouncilArea', 'Regionname']

label_encoders_melbourne = {}
for col in simple_categorical:
    le = LabelEncoder()
    X_melbourne_train_processed[col] = le.fit_transform(X_melbourne_train[col])
    X_melbourne_test_processed[col] = le.transform(X_melbourne_test[col])
    label_encoders_melbourne[col] = le


council_dummies_train = pd.get_dummies(X_melbourne_train['CouncilArea'], prefix='Council', dtype=int)
region_dummies_train = pd.get_dummies(X_melbourne_train['Regionname'], prefix='Region', dtype=int)
council_dummies_test = pd.get_dummies(X_melbourne_test['CouncilArea'], prefix='Council', dtype=int)
region_dummies_test = pd.get_dummies(X_melbourne_test['Regionname'], prefix='Region', dtype=int)

council_dummies_test = council_dummies_test.reindex(columns=council_dummies_train.columns, fill_value=0)
region_dummies_test = region_dummies_test.reindex(columns=region_dummies_train.columns, fill_value=0)

X_melbourne_train_processed = X_melbourne_train_processed.drop(['CouncilArea', 'Regionname'], axis=1)
X_melbourne_test_processed = X_melbourne_test_processed.drop(['CouncilArea', 'Regionname'], axis=1)

X_melbourne_train_processed = pd.concat([X_melbourne_train_processed, council_dummies_train, region_dummies_train], axis=1)
X_melbourne_test_processed = pd.concat([X_melbourne_test_processed, council_dummies_test, region_dummies_test], axis=1)

columns_to_drop_melbourne = ['Suburb', 'Address', 'SellerG', 'Date']
X_melbourne_train_processed = X_melbourne_train_processed.drop(columns=columns_to_drop_melbourne, errors='ignore')
X_melbourne_test_processed = X_melbourne_test_processed.drop(columns=columns_to_drop_melbourne, errors='ignore')

print(f"Итого: {X_melbourne_train_processed.shape[1]} признаков")

Type: 3
Method: 5
CouncilArea: 33
Regionname: 8
Итого: 53 признаков


Итоговые размер данных и типы данных:

In [11]:
print(f"Heart train: {X_heart_train_processed.shape}")
print(f"Heart test: {X_heart_test_processed.shape}")
print(f"Melbourne train: {X_melbourne_train_processed.shape}")
print(f"Melbourne test: {X_melbourne_test_processed.shape}")

print(f"Heart: {X_heart_train_processed.dtypes.unique()}")
print(f"Melbourne: {X_melbourne_train_processed.dtypes.unique()}")

Heart train: (734, 11)
Heart test: (184, 11)
Melbourne train: (10864, 53)
Melbourne test: (2716, 53)
Heart: [dtype('int64') dtype('float64')]
Melbourne: [dtype('int64') dtype('float64')]


Проведем масштабирование признаков:

In [12]:
scaler_heart = StandardScaler()
X_heart_train_scaled = scaler_heart.fit_transform(X_heart_train_processed)
X_heart_test_scaled = scaler_heart.transform(X_heart_test_processed)

scaler_melbourne = StandardScaler()
X_melbourne_train_scaled = scaler_melbourne.fit_transform(X_melbourne_train_processed)
X_melbourne_test_scaled = scaler_melbourne.transform(X_melbourne_test_processed)

#### Обучение моделей

In [13]:
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_heart_train_scaled, y_heart_train)

rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_melbourne_train_scaled, y_melbourne_train)

y_heart_pred = rf_classifier.predict(X_heart_test_scaled)
y_melbourne_pred = rf_regressor.predict(X_melbourne_test_scaled)

#### Оценка качества моделей

In [14]:
print("Классификация:")
accuracy = accuracy_score(y_heart_test, y_heart_pred)
recall = recall_score(y_heart_test, y_heart_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")

print("\nРегрессия:")
r2 = r2_score(y_melbourne_test, y_melbourne_pred)
mae = mean_absolute_error(y_melbourne_test, y_melbourne_pred)
print(f"R2: {r2:.4f}")
print(f"MAE: {mae:,.2f}")

Классификация:
Accuracy: 0.8750
Recall: 0.9020

Регрессия:
R2: 0.8126
MAE: 166,901.23


### Улучшение бейзлайна

Гипотезы:
- Увеличение числа деревьев может улучшить качество модели за счет более точного усреднения предсказаний.
- Ограничение глубины деревьев помогает бороться с переобучением. Подбор оптимального значения max_depth позволит модели лучше генерализоваться на тестовых данных.
- Увеличение min_samples_split уменьшает переобучение, требуя большего количества образцов для создания нового узла. Это может улучшить качество предсказаний.

Проблема: при проверке гипотез в разных блоках кода GridSearchCV подбирает параметры по отдельности, а при комбинировании они могут конфликтовать.
Было принято решение использовать одновременный подбор всех параметров через GridSearchCV.

In [15]:
param_grid_class = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10]
}

grid_class = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid_class,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
grid_class.fit(X_heart_train_scaled, y_heart_train)

param_grid_reg = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10]
}

grid_reg = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid_reg,
    cv=5,
    scoring='r2',
    n_jobs=-1
)
grid_reg.fit(X_melbourne_train_scaled, y_melbourne_train)

print(f"Классификация - лучшие параметры: {grid_class.best_params_}")
print(f"Регрессия - лучшие параметры: {grid_reg.best_params_}")

y_heart_pred_grid = grid_class.predict(X_heart_test_scaled)
y_melbourne_pred_grid = grid_reg.predict(X_melbourne_test_scaled)

accuracy_grid = accuracy_score(y_heart_test, y_heart_pred_grid)
recall_grid = recall_score(y_heart_test, y_heart_pred_grid)
r2_grid = r2_score(y_melbourne_test, y_melbourne_pred_grid)
mae_grid = mean_absolute_error(y_melbourne_test, y_melbourne_pred_grid)

print("\nСравнение с базовым бейзлайном:")
print("Классификация:")
print(f"Базовый - Accuracy: {accuracy:.4f}, Recall: {recall:.4f}")
print(f"Оптимизированный - Accuracy: {accuracy_grid:.4f}, Recall: {recall_grid:.4f}")
print("\nРегрессия:")
print(f"Базовый - R2: {r2:.4f}, MAE: {mae:,.2f}")
print(f"Оптимизированный - R2: {r2_grid:.4f}, MAE: {mae_grid:,.2f}")

Классификация - лучшие параметры: {'max_depth': 15, 'min_samples_split': 10, 'n_estimators': 100}
Регрессия - лучшие параметры: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 300}

Сравнение с базовым бейзлайном:
Классификация:
Базовый - Accuracy: 0.8750, Recall: 0.9020
Оптимизированный - Accuracy: 0.8804, Recall: 0.9216

Регрессия:
Базовый - R2: 0.8126, MAE: 166,901.23
Оптимизированный - R2: 0.8142, MAE: 166,210.87


Гипотезы подтверждены, метрики улучшились. Однако удалось добиться совсем незначительного улучшения.

### Имплементация алгоритма машинного обучения

Будем использовать упрощенную версию, реализация на sklearn слишком сложна.  
Релизация для регресии и классификации в виде классов:

In [17]:
class SimpleDecisionTreeClassifier:
    def __init__(self, max_depth: int = 5, min_samples_split: int = 2, max_features: int = None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.tree = None

    def _gini(self, y: np.ndarray) -> float:
        classes, counts = np.unique(y, return_counts=True)
        proportions = counts / len(y)
        return 1 - np.sum(proportions ** 2)

    def _best_split(self, X: np.ndarray, y: np.ndarray) -> Tuple[int, float]:
        best_gini = float('inf')
        best_feature = None
        best_threshold = None

        n_features = X.shape[1]
        if self.max_features is not None:
            n_selected = min(self.max_features, n_features)
        else:
            n_selected = int(np.sqrt(n_features))

        selected_features = random.sample(range(n_features), n_selected)

        for feature in selected_features:
            unique_values = np.unique(X[:, feature])
            if len(unique_values) > 10:
                thresholds = random.sample(sorted(unique_values), 10)
            else:
                thresholds = unique_values

            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue

                gini_left = self._gini(y[left_mask])
                gini_right = self._gini(y[right_mask])

                n_left = np.sum(left_mask)
                n_right = np.sum(right_mask)
                total = n_left + n_right

                weighted_gini = (n_left / total) * gini_left + (n_right / total) * gini_right

                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold, best_gini

    def _build_tree(self, X: np.ndarray, y: np.ndarray, depth: int = 0):
        n_samples = len(y)

        if (depth >= self.max_depth or
            n_samples < self.min_samples_split or
            len(np.unique(y)) == 1):
            return Counter(y).most_common(1)[0][0]

        feature, threshold, gini = self._best_split(X, y)
        if feature is None:
            return Counter(y).most_common(1)[0][0]

        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {
            'feature': feature,
            'threshold': threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    def fit(self, X: np.ndarray, y: np.ndarray):
        self.tree = self._build_tree(X, y)
        return self

    def _predict_sample(self, x: np.ndarray, tree: dict) -> int:
        if not isinstance(tree, dict):
            return tree

        if x[tree['feature']] <= tree['threshold']:
            return self._predict_sample(x, tree['left'])
        else:
            return self._predict_sample(x, tree['right'])

    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.array([self._predict_sample(x, self.tree) for x in X])



class SimpleDecisionTreeRegressor:
    def __init__(self, max_depth: int = 5, min_samples_split: int = 5, max_features: int = None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.tree = None

    def _mse(self, y: np.ndarray) -> float:
        if len(y) == 0:
            return 0
        mean = np.mean(y)
        return np.mean((y - mean) ** 2)

    def _best_split(self, X: np.ndarray, y: np.ndarray) -> Tuple[int, float]:
        best_mse = float('inf')
        best_feature = None
        best_threshold = None

        n_features = X.shape[1]
        if self.max_features is not None:
            n_selected = min(self.max_features, n_features)
        else:
            n_selected = int(np.sqrt(n_features))

        selected_features = random.sample(range(n_features), n_selected)

        for feature in selected_features:
            unique_values = np.unique(X[:, feature])
            if len(unique_values) > 10:
                thresholds = random.sample(sorted(unique_values), 10)
            else:
                thresholds = unique_values

            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask

                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue

                mse_left = self._mse(y[left_mask])
                mse_right = self._mse(y[right_mask])

                n_left = np.sum(left_mask)
                n_right = np.sum(right_mask)
                total = n_left + n_right

                weighted_mse = (n_left / total) * mse_left + (n_right / total) * mse_right

                if weighted_mse < best_mse:
                    best_mse = weighted_mse
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold, best_mse

    def _build_tree(self, X: np.ndarray, y: np.ndarray, depth: int = 0):
        n_samples = len(y)

        if (depth >= self.max_depth or
            n_samples < self.min_samples_split or
            np.var(y) < 0.01):
            return np.mean(y)

        feature, threshold, mse = self._best_split(X, y)

        if feature is None:
            return np.mean(y)

        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {
            'feature': feature,
            'threshold': threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    def fit(self, X: np.ndarray, y: np.ndarray):
        self.tree = self._build_tree(X, y)
        return self

    def _predict_sample(self, x: np.ndarray, tree: dict) -> float:
        if not isinstance(tree, dict):
            return tree

        if x[tree['feature']] <= tree['threshold']:
            return self._predict_sample(x, tree['left'])
        else:
            return self._predict_sample(x, tree['right'])

    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.array([self._predict_sample(x, self.tree) for x in X])


class SimpleRandomForestClassifier:
    def __init__(self, n_estimators: int = 10, max_depth: int = 5,
                 min_samples_split: int = 2, max_features: int = None,
                 random_state: int = 42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []
        random.seed(random_state)

    def _bootstrap_sample(self, X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X: np.ndarray, y: np.ndarray):
        self.trees = []

        for _ in range(self.n_estimators):
            X_sample, y_sample = self._bootstrap_sample(X, y)

            tree = SimpleDecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                max_features=self.max_features
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        predictions = np.array([tree.predict(X) for tree in self.trees])

        final_predictions = []
        for i in range(X.shape[0]):
            counter = Counter(predictions[:, i])
            final_predictions.append(counter.most_common(1)[0][0])

        return np.array(final_predictions)

class SimpleRandomForestRegressor:
    def __init__(self, n_estimators: int = 10, max_depth: int = 5,
                 min_samples_split: int = 5, max_features: int = None,
                 random_state: int = 42):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.random_state = random_state
        self.trees = []
        random.seed(random_state)

    def _bootstrap_sample(self, X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X: np.ndarray, y: np.ndarray):
        self.trees = []

        for _ in range(self.n_estimators):
            X_sample, y_sample = self._bootstrap_sample(X, y)

            tree = SimpleDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                max_features=self.max_features
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        predictions = np.array([tree.predict(X) for tree in self.trees])

        return np.mean(predictions, axis=0)

Посмотрим на результаты и сравним их с базовым бейзайном. Будем использовать уменьшенные значения параметров, так как с оригинальными значениями код работал бы несколько часов из-за отсутствия оптимизаций.

In [18]:
custom_rf_classifier = SimpleRandomForestClassifier(
    n_estimators=10,  # мало деревьев для скорости
    max_depth=5,
    min_samples_split=2,
    random_state=42
)

custom_rf_regressor = SimpleRandomForestRegressor(
    n_estimators=10,  # мало деревьев для скорости
    max_depth=5,
    min_samples_split=5,
    random_state=42
)


custom_rf_classifier.fit(X_heart_train_scaled, y_heart_train.values)
custom_rf_regressor.fit(X_melbourne_train_scaled, y_melbourne_train.values)

y_heart_pred_custom = custom_rf_classifier.predict(X_heart_test_scaled)
y_melbourne_pred_custom = custom_rf_regressor.predict(X_melbourne_test_scaled)

print("\nКастомные модели:")
print("Классификация:")
accuracy_custom = accuracy_score(y_heart_test, y_heart_pred_custom)
recall_custom = recall_score(y_heart_test, y_heart_pred_custom)
print(f"Accuracy: {accuracy_custom:.4f}")
print(f"Recall: {recall_custom:.4f}")

print("\nРегрессия:")
r2_custom = r2_score(y_melbourne_test, y_melbourne_pred_custom)
mae_custom = mean_absolute_error(y_melbourne_test, y_melbourne_pred_custom)
print(f"R2: {r2_custom:.4f}")
print(f"MAE: {mae_custom:,.2f}")


Кастомные модели:
Классификация:
Accuracy: 0.8587
Recall: 0.9020

Регрессия:
R2: 0.5641
MAE: 287,460.36


Ожидаемо получили результат хуже, чем у базового бейзлайна.  
Теперь добавим в модель техники из улучшенного бейзлайна:

In [19]:
custom_rf_classifier_improved = SimpleRandomForestClassifier(
    n_estimators=20,
    max_depth=15,
    min_samples_split=10,
    random_state=42
)

custom_rf_regressor_improved = SimpleRandomForestRegressor(
    n_estimators=20,
    max_depth=10,
    min_samples_split=5,
    random_state=42
)

custom_rf_classifier_improved.fit(X_heart_train_scaled, y_heart_train.values)
custom_rf_regressor_improved.fit(X_melbourne_train_scaled, y_melbourne_train.values)

y_heart_pred_custom_improved = custom_rf_classifier_improved.predict(X_heart_test_scaled)
y_melbourne_pred_custom_improved = custom_rf_regressor_improved.predict(X_melbourne_test_scaled)

print("\nУлучшенные кастомные модели:")
print("Классификация:")
accuracy_custom_improved = accuracy_score(y_heart_test, y_heart_pred_custom_improved)
recall_custom_improved = recall_score(y_heart_test, y_heart_pred_custom_improved)
print(f"Accuracy: {accuracy_custom_improved:.4f}")
print(f"Recall: {recall_custom_improved:.4f}")

print("\nРегрессия:")
r2_custom_improved = r2_score(y_melbourne_test, y_melbourne_pred_custom_improved)
mae_custom_improved = mean_absolute_error(y_melbourne_test, y_melbourne_pred_custom_improved)
print(f"R2: {r2_custom_improved:.4f}")
print(f"MAE: {mae_custom_improved:,.2f}")


Улучшенные кастомные модели:
Классификация:
Accuracy: 0.8859
Recall: 0.9118

Регрессия:
R2: 0.7365
MAE: 208,878.34


Внезапно получили очень хороший результат для классификации и хороший прирост для регрессии. Вероятно, увеличение глубины деревьев позволило модели уловить более сложные закономерности.

### Выводы

- Библиотечные реализации через sklearn показали высокое качество на обоих датасетах.
- Упрощенная собственная реализация уступает по точности, но демонстрирует работоспособность алгоритма.
- Применение техник из улучшенного бейзлайна дало значительный прирост качества для кастомных моделей.
- Для классификации улучшенная кастомная модель даже превзошла оптимизированную sklearn-модель.
- Подбор гиперпараметров остается важным этапом независимо от реализации.